# Expected Shot Value 
## Spatial Modeling for NBA Shots

We should expect that two important drivers of the likelihood of making a shot are the shot distance and how near the closest defender is.  Using data obtained from NBA.com for the 2014-2015 season, this demo explores how you can build an expected value model for shooting.

## Setup and Load Data

In [ ]:
%run ../../utils/notebook_setup.py

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

from datascience_utils import hexbin_plot, sorted_boxplot
from datascience_topic import build_expected_value_from_hexbin

In [ ]:
df = pd.read_csv('shot_logs_2014_15.csv.gz')

In [ ]:
df.head()

In [ ]:
df.info()

## Defender Distance
There are lots of things you could do with this data but let's just jump right in with defender distance.

We use the 1ft buckets in `CLOSE_DEF_DIST_ROUNDED` to compute FG% as a function of defender distance.

After grouping, we plot the relation and... wait, why does defender distance not seem to matter!?

_Question_
+ What might explain this seeming lack of relationship?  Does shot type matter?  Anything else?

In [ ]:
result = df.groupby('CLOSE_DEF_DIST_ROUNDED').\
    agg({'SHOT_RESULT_BIN': np.mean})

fig, ax = plt.subplots()
result.loc[:14].plot(label="All Shots", ax=ax)
ax.set_ylabel('Shooting Pct')
ax.set_xlabel("Closest Defender Distance (ft)")
ax.set_ylim(0, .7)
ax.legend();

Okay, now we separate by shot type.  Clearly defender distance is a big deal for 3pt shots.  But it still doesn't seem that important for 2pt shots.

_Question_
+ Should 2pt shooting % be immune defender distance?  Why or why not?  Is there something else missing?

In [ ]:
result = df.groupby(['PTS_TYPE', 'CLOSE_DEF_DIST_ROUNDED']).\
    agg({'SHOT_RESULT_BIN': np.mean})
    
fig, ax = plt.subplots()
result.loc[2].loc[:14].plot(label='2pt', ax=ax)
result.loc[3].loc[:14].plot(label='3pt', ax=ax)
ax.set_ylabel('Shooting Pct')
ax.set_xlabel("Closest Defender Distance (ft)")
ax.set_ylim(0, .7)
ax.legend();

## Shot Distance

We can quickly visualize the variability in shot distance with a pair of histogram plots.  The distance of the NBA 3pt line is not uniform but generally you see the overall pattern of shots.

In [ ]:
nbins = 150
alpha = .5

dist_mask = df['SHOT_DIST'] <= 30
twopt = df['PTS_TYPE'] == 2
ax = df.loc[twopt & dist_mask].hist('SHOT_DIST', bins=nbins, alpha=alpha)

threept = df['PTS_TYPE'] == 3
df.loc[threept & dist_mask].hist('SHOT_DIST', bins=nbins, color='C1', alpha=alpha, ax=ax);

We can group by shot type and shot distance (rounded to nearest half foot) to get a feel for how shooting percentage varies with shot distance.  Obviously close in shots are layups/dunks that are almost always converted.

In [ ]:
result = df.groupby(['PTS_TYPE', 'SHOT_DIST_ROUNDED']).\
    agg({'SHOT_RESULT_BIN': np.mean}).\
    sort_index(level=0)

fig, ax = plt.subplots()
result.loc[2].loc[:24].plot(label='2pt', ax=ax)
result.loc[3].loc[21:29].plot(label='3pt', ax=ax)
ax.set_ylabel('Shooting Pct')
ax.set_xlabel("Shot Distance (ft)")
ax.set_ylim(0, .7)
ax.legend();

## Shot Distance and Defender Distance

Okay, we saw defender distance mattered for 3s and shot distance matters.  Let's combine the two.  We use the shot distance buckets in `SHOT_DIST_BUCKET` as well as the rounded defender distance to show how shooting percentage varies both in shot distance and defender distance.

In [ ]:
result = df.groupby(['SHOT_DIST_BUCKET', 'CLOSE_DEF_DIST_ROUNDED']).\
    agg({'SHOT_RESULT_BIN': np.mean})

shot_buckets = ['0-3', '3-10', '10-16', '16-3pt', '3pt']
fig, ax = plt.subplots()
for bucket in shot_buckets:
    result.loc[bucket].loc[:14].plot(label=bucket, ax=ax)

ax.set_ylabel('Shooting Pct')
ax.set_xlabel("Closest Defender Distance (ft)")
ax.set_ylim(0, 1.03)
ax.legend(bbox_to_anchor=(1.02, 1.02));

### Hexbin Plot

We can view the relationship between the in more continuous space with a hexbin plot.  A hexbin plot segments the space and all shots that lie within a hexagon will be grouped together.  Then within that hexagon, we will compute the shooting percentage.  We can see how regardless of shot distance, the defender distance near 0 drives down shooting percentage at that distance from the basket.

_Question_
+ Why are there some spots that are extra dark or white that don't really fit with the general pattern?
+ What is going on with that spike on the left side?  What kind of shot has a high percentage, is near the basket, and has no defender nearby?

In [ ]:
hexbin_fgpct_plot = hexbin_plot(
    df.loc[(df.SHOT_DIST <= 30)],
    'SHOT_DIST',
    'CLOSE_DEF_DIST',
    C='SHOT_RESULT_BIN',
    collect=np.mean,
    gridsize=20,
    figsize=(8, 6),
    cmap='viridis',
    mincnt=5,
    vmin=0.2,
    vmax=0.8
)
hexbin_fgpct_plot.tick_params(reset=True)
hexbin_fgpct_plot.set_ylabel("Defender Dist (ft)")
hexbin_fgpct_plot.set_xlabel("Shot Dist (ft)")
cax = plt.gcf().get_axes()[1]
cax.set_yticklabels(['< 0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '> 0.8']);

## Expected Shot Value

We can do the same hexbin plot but compute (approximate) expected points.  This makes more sense due to the 3pt line and represents a shooting efficiency due to the variable value of shots.  Note that since some 3pt shots are in the corner and thus < 23' 9" away from the basket, this is only approximate.  There's a better way to compute ESV.

_Question_
+ What does this hexbin plot say about mid/long-distance 2s as well?  How bad is a closely guarded mid/long 2 point shot?

In [ ]:
hexbin_esv_plot = hexbin_plot(
    df.loc[(df.SHOT_DIST <= 30)],
    'SHOT_DIST',
    'CLOSE_DEF_DIST',
    C='PTS_MADE',
    collect=np.mean,
    gridsize=20,
    figsize=(8, 6),
    cmap='viridis',
    mincnt=5,
    vmin=0.5,
    vmax=1.6
)
hexbin_esv_plot.tick_params(reset=True)
hexbin_esv_plot.set_ylabel("Defender Dist (ft)")
hexbin_esv_plot.set_xlabel("Shot Dist (ft)")
cax = plt.gcf().get_axes()[1]
cax.set_yticklabels(['< 0.6', '0.8', '1.0', '1.2', '1.4', '> 1.6']);

## An Improved Shooting Metric?

Recall the Effective Field Goal Pct was given by
$$
    \text{eFG%} = \frac{\mathit{FG} + .5 \cdot \mathit{3FG}}{\mathit{FGA}} = \frac{\text{Total Points (excluding FT)}}{2 \cdot \mathit{FGA}}
$$

Ignoring the division by 2, our ESV computation is akin to the EFG computation: expected points scored per attempt.

Here's a metric we can build along what we've seen already in baseball: for each shot use the hexbin plot to compute the expected FG% which we use to compute the expected shot value:  
$$
    \text{eSV} = \text{Point Value} \times \text{Expected FG% at Distance $R$ and Defender Dist. $X$}
$$

We can also compute the points above average for each shot as
$$
    \text{Points Above Average} = \text{Points Made} - \text{eSV}
$$

This is actually akin to what was proposed in this [paper][1] at the Sloan conference.

[1]: http://www.sloansportsconference.com/wp-content/uploads/2014/02/2014-SSAC-Quantifying-Shot-Quality-in-the-NBA.pdf

In [ ]:
# Use a helper function to get the bin (and therefore FG%) for each shot
x_fgpct = build_expected_value_from_hexbin(
    df['SHOT_DIST'], df['CLOSE_DEF_DIST'], hexbin_fgpct_plot)

df['EXPECTED_FG%'] = x_fgpct
df['EXPECTED_SHOT_VALUE'] = x_fgpct * df['PTS_TYPE']

df['EXPECTED_SHOT_VALUE'].hist(bins=50);

In [ ]:
df['PTS_ABOVE_AVG'] = df['PTS_MADE'] - df['EXPECTED_SHOT_VALUE']
ax = df.hist(column='PTS_ABOVE_AVG', bins=50);

### Player ESV
Which players are taking the highest value shots?  How about the lowest?

In [ ]:
player_esv = df.groupby('playerName').\
    agg({
        'EXPECTED_SHOT_VALUE': np.mean, 
        'PTS_MADE': np.mean, 
        "SHOT_RESULT": len
    }).\
    rename({
        'EXPECTED_SHOT_VALUE': 'ESVperFGA',
        'PTS_MADE': 'PTSperFGA',
        'SHOT_RESULT': 'FGA'
    }, axis=1)

In [ ]:
player_esv.loc[player_esv.FGA > 100].\
    sort_values('ESVperFGA', ascending=False).\
    head(20)

In [ ]:
player_esv.loc[player_esv.FGA > 100].\
    sort_values('ESVperFGA', ascending=True).\
    head(20)

In [ ]:
fig, ax = plt.subplots()
player_esv.loc[player_esv.FGA > 100].\
    plot.scatter('PTSperFGA', 'ESVperFGA', ax=ax)
ax.set_aspect('equal')
ax.plot([.8, 1.2], [.8, 1.2], color='C1');

### Team ESV per 88 FGA 

Which teams take the best shots?  88 FGA is about league average for 100 possessions.

In [ ]:
team_esv = df.groupby('PLAYER_TEAM').\
    agg({
        'EXPECTED_SHOT_VALUE': np.mean, 
        'PTS_MADE': np.mean, 
        "SHOT_RESULT": len
    }).\
    rename({
        'EXPECTED_SHOT_VALUE': 'ESVperFGA',
        'PTS_MADE': 'PTSperFGA',
        'SHOT_RESULT': 'FGA'
    }, axis=1)
team_esv['ESVper88FGA'] = 88 * team_esv['ESVperFGA']

team_esv.sort_values('ESVper88FGA', ascending=False)

In [ ]:
fig, ax = plt.subplots()
team_esv.plot.scatter('PTSperFGA', 'ESVperFGA', ax=ax)
ax.set_aspect('equal')
ax.plot([.9, 1.1], [.9, 1.1], color='C1');

### Player Points Above Average per FGA
We can compute Points Above Average per FGA for each player.

In [ ]:
player_esv['PTS_ABOVE_AVG_perFGA'] = player_esv['PTSperFGA'] - player_esv['ESVperFGA']

In [ ]:
player_esv.loc[player_esv.FGA > 100].\
    sort_values('PTS_ABOVE_AVG_perFGA', ascending=False).\
    head(20)

In [ ]:
player_esv.loc[player_esv.FGA > 100].\
    sort_values('PTS_ABOVE_AVG_perFGA', ascending=False).\
    tail(20)

### Team Points Above Average per 85 FGA 
We can compute PAA per 85 FGA for each team.  85 FGA is about 1 game.

In [ ]:
team_esv['PTS_ABOVE_AVG_perFGA'] = team_esv['PTSperFGA'] - team_esv['ESVperFGA']
team_esv['PTS_ABOVE_AVG_per88FGA'] = 88 * team_esv['PTS_ABOVE_AVG_perFGA']

team_esv.sort_values('PTS_ABOVE_AVG_per88FGA', ascending=False)

## Does Defender Distance Relate to Shooter Quality?

Presumably better shooters will be guarded more closely, right?  To study this, let's restrict to catch & shoot 3pt shots.  We'll define catch and shoot shots as those with 0 dribbles and a touch time of less than 2 seconds.

In [ ]:
catch_and_shoot = (df.TOUCH_TIME >= 0) & (df.TOUCH_TIME < 2) & (df.DRIBBLES == 0)

In [ ]:
steph = df.playerName == 'Stephen Curry'
fig, ax = plt.subplots()
df.loc[steph & threept & catch_and_shoot].hist('CLOSE_DEF_DIST', bins=30, ax=ax)
ax.set_xlim(0, 25)
ax.set_title("Closest Defender Dist (ft) - Steph Curry C&S 3FGA");

We take all players with at least 50 shots, sort them by their 3FG% (from left to right), and construct boxplots showing defender distance.  If 3FG% indicates better shooters, and it should, then those players should be guarded tighter than poor shooters. We should see some kind of pattern in the plot that will reflect this hypothesis.

_Question_
+ What pattern emerges from this plot?  Can you think of a plausible explanation? 

In [ ]:
num_3pt_shots = 100
by = 'playerName'
column = "CLOSE_DEF_DIST"

c_and_s_three_pt_shots = df.loc[threept & catch_and_shoot]

by_3pt_pct = c_and_s_three_pt_shots.\
    groupby(by).\
    agg({'SHOT_RESULT_BIN': np.mean, 'SHOT_RESULT': len}).\
    rename({'SHOT_RESULT_BIN': '3FG%', 'SHOT_RESULT': '3FGA'}, axis=1).\
    sort_values(by='3FG%', ascending=True)

by_3pt_pct = by_3pt_pct.loc[by_3pt_pct['3FGA'] > num_3pt_shots]
player_order = by_3pt_pct.index

In [ ]:
sorted_boxplot(c_and_s_three_pt_shots, by, column, player_order, figsize=(25, 10), fontsize=8)